In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'
final_result = '/content/drive/MyDrive/my_project/my_project1/result/reg'
final_img = '/content/drive/MyDrive/my_project/my_project1/result/img'
# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# # 1. 나눔 폰트 다운
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
import pandas as pd
import numpy as np

import geopandas as gpd

import statsmodels.api as sm
import re
from io import StringIO

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [4]:
year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'temp_{year}'] = pd.read_csv(f'{data_path}/final/final_{year}.csv')
    globals()[f'temp_{year}'] = globals()[f'temp_{year}'].replace('-', '0')



In [5]:
depend_list = ['solo_all_ratio', 'solo_20s_ratio', 'solo_30s_ratio']
depend_name = ['all', '20', '30']

In [6]:
var_col = ['네일숍', '미용실', '애완동물',
           '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점', '치킨전문점',
           '커피-음료', '패스트푸드점', '제과점', '호프-간이주점',
            '대형마트', '쇼핑업체수', '편의점', '슈퍼마켓'
           ]

nocal_col = ['사업체수', '종사자수', '지가변동률', '지가', '대학교', '기타대학' ]

drop_col = [ '면적', '월평균소득', '총상주인구', '20대상주인구', '총직장인구', '20대직장인구', '유통업체수',]

reg_col = ['ADM_CD'] + depend_list + var_col + nocal_col

In [7]:
temp_2022.head()

,ADM_CD,total_all_2022,solo_all_2022,total_20_2022,solo_20_2022,total_25_2022,solo_25_2022,total_30_2022,solo_30_2022,total_35_2022,...,총매출금액_양식음식점_2022,총매출금액_일식음식점_2022,총매출금액_제과점_2022,총매출금액_중식음식점_2022,총매출금액_치킨전문점_2022,총매출금액_커피-음료_2022,총매출금액_패스트푸드점_2022,총매출금액_편의점_2022,총매출금액_한식음식점_2022,총매출금액_호프-간이주점_2022
0,11010530,7848.0,1854.0,582.0,183.0,687.0,220.0,779.0,244.0,758.0,...,2.700544e+09,4.009334e+09,1.024866e+09,2.067096e+09,151123891.0,8.715381e+09,749006078.0,4.259033e+09,2.279076e+10,1.627539e+09
1,11010540,2384.0,831.0,156.0,78.0,172.0,90.0,193.0,90.0,211.0,...,3.243627e+09,2.692862e+07,6.108442e+08,0.000000e+00,0.0,4.847156e+09,7462031.0,4.966037e+08,4.833707e+09,1.334099e+08
2,11010550,9116.0,1530.0,1147.0,201.0,841.0,154.0,711.0,181.0,652.0,...,1.901209e+08,0.000000e+00,2.658687e+08,4.502295e+07,27441593.0,4.592954e+08,107221827.0,7.261664e+08,5.864678e+08,1.911793e+07
3,11010560,15369.0,1604.0,1269.0,132.0,1173.0,127.0,1172.0,144.0,1075.0,...,1.062373e+08,1.879954e+07,3.815037e+08,2.886612e+08,0.0,5.489234e+08,0.0,4.087258e+08,2.379329e+09,1.762930e+08
4,11030740,18531.0,5332.0,1138.0,439.0,1829.0,748.0,2393.0,897.0,2037.0,...,8.993222e+09,1.640832e+09,5.732351e+08,9.877729e+08,770611211.0,2.705864e+09,228290638.0,3.617802e+09,1.480354e+10,2.165285e+09


In [8]:
for year in year_list:
    df = globals()[f'temp_{year}'].copy()

    # df 칼럼 정리
    # 변수명 뒤에 연도 써있는 변수들 제거
    columns_2022 = [col.replace('_2022', '') for col in df.columns]
    columns_2023 = [col.replace('_2023', '') for col in columns_2022]
    columns_2024 = [col.replace('_2024', '') for col in columns_2023]
    df.columns = columns_2024
    globals()[f'df_{year}'] = df.copy()

    reg = df[reg_col + ['total_all']]
    # 인구 1만명당 변수 확인
    for var in var_col:
        reg[var] = reg[var] / reg['total_all'] * 10000

    reg.drop(columns = 'total_all', inplace=True)
    globals()[f'reg_{year}'] = reg.copy()

In [9]:
df_2023.head()

,ADM_CD,total_all,solo_all,total_20,solo_20,total_25,solo_25,total_30,solo_30,total_35,...,총매출금액_양식음식점,총매출금액_일식음식점,총매출금액_제과점,총매출금액_중식음식점,총매출금액_치킨전문점,총매출금액_커피-음료,총매출금액_패스트푸드점,총매출금액_편의점,총매출금액_한식음식점,총매출금액_호프-간이주점
0,11010530,7845.0,1869.0,583.0,172.0,686.0,241.0,778.0,222.0,757.0,...,5.351868e+09,5.112117e+09,1.203470e+09,2.749773e+09,673323360.0,1.187346e+10,902523398.0,5.084405e+09,3.660644e+10,3.245160e+09
1,11010540,2370.0,800.0,143.0,68.0,172.0,85.0,193.0,92.0,211.0,...,4.447327e+09,2.441660e+07,1.178688e+09,0.000000e+00,0.0,6.900190e+09,111870248.0,5.324073e+08,7.375948e+09,2.476535e+08
2,11010550,9116.0,1549.0,1147.0,213.0,841.0,154.0,711.0,187.0,652.0,...,3.769157e+08,0.000000e+00,3.140562e+08,5.614901e+07,0.0,4.776125e+08,189999970.0,8.978345e+08,7.587584e+08,2.295424e+07
3,11010560,15369.0,1617.0,1269.0,136.0,1173.0,123.0,1172.0,143.0,1075.0,...,7.416936e+07,1.221249e+07,3.463709e+08,2.517208e+08,0.0,9.633955e+08,284394280.0,3.962739e+08,2.456528e+09,2.872706e+08
4,11030740,18463.0,5314.0,1138.0,442.0,1823.0,743.0,2384.0,881.0,2035.0,...,1.119020e+10,1.989825e+09,5.286337e+08,1.512627e+09,910011062.0,4.049663e+09,415945610.0,4.542987e+09,2.130260e+10,3.918752e+09


In [10]:
reg_2023.head()

,ADM_CD,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,11010530,23.824092,11.048689,12.092028,16.571064,40.790312,10.197578,158.062460,527.724665,231.994901,...,0.0,0.0,14.021670,0.000000,4994,56046,7.902167,98.973167,0.0,0.0
1,11010540,33.755274,9.562500,11.437500,8.438819,0.000000,0.000000,71.729958,308.016878,358.649789,...,0.0,0.0,4.219409,0.000000,970,5118,7.840571,99.057857,0.0,0.0
2,11010550,16.992102,11.846352,10.942544,0.000000,10.969724,7.678806,15.357613,41.684950,40.587977,...,0.0,0.0,4.387889,0.000000,908,4140,8.004000,98.822000,1.0,0.0
3,11010560,10.521179,8.008658,8.163265,1.301321,10.410567,1.951981,9.109246,35.786323,11.711888,...,0.0,0.0,1.951981,0.000000,1286,4294,8.009000,98.655000,0.0,0.0
4,11030740,28.781888,11.149793,14.904027,6.499485,24.914694,6.499485,14.082218,109.408005,117.532362,...,0.0,0.0,5.957862,1.624871,3213,20440,10.155000,98.676000,0.0,0.0


# 데이터 정규화

In [11]:
### 데이터 정규화

year_list = ['2022', '2023', '2024']

for year in year_list:
    globals()[f'reg_{year}'] = globals()[f'reg_{year}'].replace('-', '0')

    std_scaler = StandardScaler()
    df_std = std_scaler.fit_transform(globals()[f'reg_{year}'])
    df_std = pd.DataFrame(df_std)
    df_std.columns = globals()[f'reg_{year}'].columns
    df_std.drop(columns = 'ADM_CD', inplace = True)

    globals()[f'reg_{year}'] = df_std.copy()



In [12]:
reg_2022.head()

,solo_all_ratio,solo_20s_ratio,solo_30s_ratio,네일숍,미용실,애완동물,분식전문점,한식음식점,양식음식점,일식음식점,...,대형마트,쇼핑업체수,편의점,슈퍼마켓,사업체수,종사자수,지가변동률,지가,대학교,기타대학
0,0.415566,0.085282,0.198901,0.056452,-0.709275,-0.348807,2.276832,1.256923,2.508037,1.311383,...,-0.301346,-0.306233,-0.454019,-0.329860,0.814002,1.957975,-0.003407,-0.441216,-0.288565,-0.22305
1,1.502040,-0.042339,-0.003637,-0.887900,-1.308291,-1.355888,0.531416,0.342283,4.130139,0.129319,...,-0.301346,-0.306233,-0.717320,-0.329860,-0.669465,-0.393207,-0.222565,-0.487585,-0.288565,-0.22305
2,-0.245995,0.208340,-0.120176,-1.208124,-1.136392,-0.835688,-0.547488,-0.743577,-0.024359,-0.595684,...,-0.301346,-0.306233,-0.955882,-0.329860,-0.692322,-0.438358,0.513886,-0.239215,2.543395,-0.22305
3,-0.859866,-0.383977,-0.892781,-1.108779,-1.145155,-1.201612,-0.655428,-0.779071,-0.395495,-0.628621,...,-0.301346,-0.306233,-1.018533,-0.329860,-0.552970,-0.431248,0.343233,-0.503533,-0.288565,-0.22305
4,0.913616,0.129359,1.004618,-0.754961,-0.919304,-0.758780,-0.562672,-0.488637,0.932041,-0.049448,...,-0.301346,-0.306233,-0.832113,1.569775,0.157428,0.314161,1.499884,-1.386783,-0.288565,-0.22305


In [13]:
selected_var = reg_2022.columns[len(depend_list):]
selected_var

Index(['네일숍', '미용실', '애완동물', '분식전문점', '한식음식점', '양식음식점', '일식음식점', '중식음식점',
       '치킨전문점', '커피-음료', '패스트푸드점', '제과점', '호프-간이주점', '대형마트', '쇼핑업체수', '편의점',
       '슈퍼마켓', '사업체수', '종사자수', '지가변동률', '지가', '대학교', '기타대학'],
      dtype='object')

In [14]:
### 분석 - OLS

df_ols = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}']
    for idx, depend in enumerate(depend_list):
        X = sm.add_constant(df[selected_var])

        # Fit the OLS model
        model = sm.OLS(df[depend], X)
        result = model.fit()

        summary = result.summary()

        temp = summary.tables[1].as_csv()

        ols_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        ols_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']
        ols_result = ols_result[['var', 'coef', 'p']]
        ols_result['p'] = ols_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        ols_result['coef'] = ols_result['coef'].apply(lambda x: round(x, 3))
        ols_result.columns = [f'var', f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        ols_result['var'] = ols_result['var'].str.strip()

        globals()[f'ols_{year}_{depend}'] = ols_result

        if (year == '2022') & (idx == 0):
            df_ols = ols_result
        else:
            df_ols = df_ols.merge(ols_result,
                                        how = 'left',
                                        on = 'var')

In [15]:
ols_2022_solo_30s_ratio

,var,c_2022_30,p_2022_30
0,const,-0.000,
1,네일숍,0.377,+
2,미용실,-0.074,
3,애완동물,-0.060,
4,분식전문점,-0.043,
5,한식음식점,-0.487,+
6,양식음식점,0.084,
7,일식음식점,0.610,+
8,중식음식점,-0.013,
9,치킨전문점,-0.245,+


In [16]:
df_ols.to_csv(f'{final_result}/ols_all.csv', index = False)
df_ols.to_excel(f'{final_result}/ols_all.xlsx', index = False)

## 컬럼명을 영어로 바꾸는 준비
- 전진선택법, 후진제거법은 변수명 영어로

In [17]:
reg_2022_en = reg_2022.copy()

# 종속변수에 해당하는 것은 영어 이름
# 독립변수만 var_ 로 바꿔준다
new_columns = ['var{}'.format(i) for i in range(1,len(reg_col) - len(depend_list) )]
reg_2022_en.columns = depend_list + new_columns
reg_2022_en.columns

Index(['solo_all_ratio', 'solo_20s_ratio', 'solo_30s_ratio', 'var1', 'var2',
       'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var10',
       'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17', 'var18',
       'var19', 'var20', 'var21', 'var22', 'var23'],
      dtype='object')

In [18]:
## 결과표에서 한글로 보기 위해
# 변수명을 dictionary로
col_dict = {}

for i in range(len(df_std.columns)):
    col_dict[reg_2022_en.columns[i]] = reg_2022.columns[i]
    col_dict[reg_2022.columns[i]] = reg_2022_en.columns[i]

col_dict.update({'Intercept' : '상수',
                 '상수' : 'intercept',
                 'const' : '상수',
                 '상수' : 'const'})

col_dict

{'solo_all_ratio': 'solo_all_ratio',
 'solo_20s_ratio': 'solo_20s_ratio',
 'solo_30s_ratio': 'solo_30s_ratio',
 'var1': '네일숍',
 '네일숍': 'var1',
 'var2': '미용실',
 '미용실': 'var2',
 'var3': '애완동물',
 '애완동물': 'var3',
 'var4': '분식전문점',
 '분식전문점': 'var4',
 'var5': '한식음식점',
 '한식음식점': 'var5',
 'var6': '양식음식점',
 '양식음식점': 'var6',
 'var7': '일식음식점',
 '일식음식점': 'var7',
 'var8': '중식음식점',
 '중식음식점': 'var8',
 'var9': '치킨전문점',
 '치킨전문점': 'var9',
 'var10': '커피-음료',
 '커피-음료': 'var10',
 'var11': '패스트푸드점',
 '패스트푸드점': 'var11',
 'var12': '제과점',
 '제과점': 'var12',
 'var13': '호프-간이주점',
 '호프-간이주점': 'var13',
 'var14': '대형마트',
 '대형마트': 'var14',
 'var15': '쇼핑업체수',
 '쇼핑업체수': 'var15',
 'var16': '편의점',
 '편의점': 'var16',
 'var17': '슈퍼마켓',
 '슈퍼마켓': 'var17',
 'var18': '사업체수',
 '사업체수': 'var18',
 'var19': '종사자수',
 '종사자수': 'var19',
 'var20': '지가변동률',
 '지가변동률': 'var20',
 'var21': '지가',
 '지가': 'var21',
 'var22': '대학교',
 '대학교': 'var22',
 'var23': '기타대학',
 '기타대학': 'var23',
 'Intercept': '상수',
 '상수': 'const',
 'const': '상수'}

# 전진선택법

In [19]:
### 전진선택법 함수

def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
        else:
            break
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model


In [20]:
### 분석 - fwd

df_fwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()
    df.columns = depend_list + new_columns

    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = forward_selection(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        fwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        fwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        fwd_result['var'] = fwd_result['var'].str.strip()
        fwd_result['coef'] = fwd_result['coef'].apply(lambda x: round(x, 3))
        fwd_result['var_mapped'] = fwd_result['var'].apply(lambda x: col_dict[x])
        fwd_result = fwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        fwd_result['var_mapped'] = fwd_result['var_mapped'].str.strip()
        fwd_result['p'] = fwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        fwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']
        globals()[f'fwd_{year}_{depend}'] = fwd_result

        if (year == '2022') & (idx == 0):
            df_fwd = fwd_result
        else:
            df_fwd = df_fwd.merge(fwd_result,
                                        how = 'left',
                                        on = ['var', 'var2'])

In [21]:
df_fwd.to_csv(f'{final_result}/fwd_all.csv', index = False)
df_fwd.to_excel(f'{final_result}/fwd_all.xlsx', index = False)

In [22]:
df_fwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,Intercept,상수,0.000,,0.000,,-0.000,,0.000,,0.000,,0.000,,0.000,,0.000,,-0.000,
1,var6,양식음식점,0.306,+,0.146,+,NaN,NaN,0.252,+,NaN,NaN,0.156,+,0.227,+,0.163,+,0.143,+
2,var18,사업체수,0.443,+,0.167,+,0.256,+,0.517,+,0.151,+,0.359,+,0.539,+,0.309,+,0.320,+
3,var22,대학교,0.165,+,0.436,+,-0.079,+,0.165,+,0.433,+,-0.082,+,0.164,+,0.441,+,-0.063,
4,var4,분식전문점,0.213,+,NaN,NaN,NaN,NaN,0.192,+,NaN,NaN,NaN,NaN,0.308,+,NaN,NaN,-0.035,
5,var12,제과점,-0.271,+,NaN,NaN,-0.404,+,-0.320,+,NaN,NaN,-0.334,+,-0.203,+,-0.157,,-0.310,+
6,var3,애완동물,-0.523,+,-0.444,+,NaN,NaN,-0.664,+,-0.636,+,NaN,NaN,-0.584,+,-0.594,+,NaN,NaN
7,var2,미용실,0.354,+,0.362,+,NaN,NaN,0.486,+,0.566,+,-0.340,+,0.211,,0.054,,NaN,NaN
8,var16,편의점,0.305,+,NaN,NaN,0.325,+,0.430,+,0.311,+,0.470,+,0.397,+,0.349,+,0.278,+
9,var19,종사자수,-0.189,,NaN,NaN,NaN,NaN,-0.259,+,NaN,NaN,-0.085,,-0.307,+,-0.253,+,NaN,NaN


### 후진제거법

In [23]:
def backward_elimination(data, response, significance_level=0.05):
    features = data.columns.tolist()
    features.remove(response)

    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])  # Add a constant term for the intercept
        model = sm.OLS(data[response], features_with_constant).fit()
        p_values = model.pvalues.iloc[1:]  # Get p-values for the features only
        max_p_value = p_values.max()  # Find the feature with the maximum p-value
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break

    model = sm.OLS(data[response], sm.add_constant(data[features])).fit()
    return model


In [24]:
### 분석 - bwd
df_bwd = pd.DataFrame()

for year in year_list:
    df = globals()[f'reg_{year}'].copy()

    df.columns = depend_list + new_columns


    for idx, depend in enumerate(depend_list):
        drop_list = [item for item in depend_list if item != depend]

        model = backward_elimination(df.drop(columns = drop_list), depend)
        summary = model.summary()
        temp = summary.tables[1].as_csv()

        bwd_result = pd.read_csv(StringIO(temp), skipinitialspace=True)
        bwd_result.columns = ['var', 'coef', 'std', 't', 'p', 'confi25', 'conf97']

        bwd_result['var'] = bwd_result['var'].str.strip()
        bwd_result['coef'] = bwd_result['coef'].apply(lambda x: round(x, 3))
        bwd_result['var_mapped'] = bwd_result['var'].apply(lambda x: col_dict[x])
        bwd_result = bwd_result[['var', 'var_mapped', 'coef', 'p' ]]
        bwd_result['var_mapped'] = bwd_result['var_mapped'].str.strip()
        bwd_result['p'] = bwd_result['p'].apply(lambda x: '+' if x < 0.05 else ' ')
        bwd_result.columns = [f'var2', f'var',f'c_{year}_{depend_name[idx]}', f'p_{year}_{depend_name[idx]}']

        globals()[f'bwd_{year}_{depend}'] = bwd_result

        if (year == '2022') & (idx == 0):
            df_bwd = bwd_result
        else:
            df_bwd = df_bwd.merge(bwd_result,
                                        how = 'left',
                                       on = ['var', 'var2'])

In [25]:
df_bwd.to_csv(f'{final_result}/bwd_all.csv', index = False)
df_bwd.to_excel(f'{final_result}/bwd_all.xlsx', index = False)

In [26]:
df_bwd

,var2,var,c_2022_all,p_2022_all,c_2022_20,p_2022_20,c_2022_30,p_2022_30,c_2023_all,p_2023_all,c_2023_20,p_2023_20,c_2023_30,p_2023_30,c_2024_all,p_2024_all,c_2024_20,p_2024_20,c_2024_30,p_2024_30
0,const,상수,0.000,,0.000,,-0.000,,0.000,,0.000,,0.000,,0.000,,0.000,,-0.000,
1,var2,미용실,0.337,+,NaN,NaN,NaN,NaN,0.347,+,NaN,NaN,-0.409,+,NaN,NaN,NaN,NaN,NaN,NaN
2,var3,애완동물,-0.517,+,-0.388,+,NaN,NaN,-0.625,+,-0.465,+,NaN,NaN,-0.542,+,-0.596,+,NaN,NaN
3,var4,분식전문점,0.199,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.359,+,NaN,NaN,NaN,NaN
4,var6,양식음식점,0.302,+,0.138,+,NaN,NaN,0.337,+,0.181,+,0.166,+,0.223,+,0.131,+,0.143,+
5,var12,제과점,-0.332,+,-0.257,+,-0.373,+,-0.315,+,-0.292,+,-0.298,+,-0.279,+,NaN,NaN,-0.305,+
6,var16,편의점,0.287,+,0.296,+,0.275,+,0.513,+,0.511,+,0.400,+,0.461,+,0.329,+,NaN,NaN
7,var18,사업체수,0.448,+,0.111,+,0.274,+,0.557,+,0.105,+,0.309,+,0.536,+,0.346,+,0.316,+
8,var19,종사자수,-0.208,+,NaN,NaN,NaN,NaN,-0.296,+,NaN,NaN,NaN,NaN,-0.344,+,-0.263,+,NaN,NaN
9,var22,대학교,0.165,+,0.447,+,-0.082,+,0.169,+,0.451,+,-0.088,+,0.166,+,0.438,+,NaN,NaN


# 각연도별 통합

In [27]:
for year in year_list:

    for idx, depend in enumerate(depend_list):
        ols = globals()[f'ols_{year}_{depend}']
        fwd = globals()[f'fwd_{year}_{depend}']
        bwd = globals()[f'bwd_{year}_{depend}']

        ols['var'] = ols['var'].apply(lambda x: x if x != 'const' else '상수')

        temp = ols.merge(fwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_ols', '_fwd'))
        temp = temp.merge(bwd.drop(columns = 'var2'),
                    how = 'outer',
                    on = 'var',
                    suffixes=('_fwd', '_bwd'))

        temp = temp.sort_values(by=[f'c_{year}_{depend_name[idx]}_fwd', f'p_{year}_{depend_name[idx]}'])
        temp = temp[temp['var'] != '상수']

        globals()[f'result_{year}_{depend}']= temp

        globals()[f'result_{year}_{depend}'].to_csv(f'{final_result}/{year}_{depend}.csv', index = False)
        globals()[f'result_{year}_{depend}'].to_excel(f'{final_result}/{year}_{depend}.xlsx', index = False)

        globals()[f'result_{year}_{idx}'] = temp

In [28]:
result_2022_1


,var,c_2022_20_ols,p_2022_20_ols,c_2022_20_fwd,p_2022_20_fwd,c_2022_20,p_2022_20
3,애완동물,-0.461,+,-0.444,+,-0.388,+
15,쇼핑업체수,-0.157,+,-0.119,+,-0.122,+
23,기타대학,0.083,,0.082,,NaN,NaN
20,지가변동률,0.131,,0.098,+,0.103,+
6,양식음식점,0.121,,0.146,+,0.138,+
18,사업체수,0.323,+,0.167,+,0.111,+
2,미용실,0.015,,0.362,+,NaN,NaN
22,대학교,0.437,+,0.436,+,0.447,+
1,네일숍,0.291,+,NaN,NaN,0.256,+
12,제과점,-0.192,,NaN,NaN,-0.257,+
